In [2]:
import os
import numpy as np
import json
import requests

In [2]:
def check_if_r(oid):
    bands = {'1':'g', '2':'r', '3':'i'}
    str_oid = str(oid)
    if len(str_oid) != 15:
        return True if bands[str_oid[4]]=='r' else False

    else:
        return True if bands[str_oid[3]]=='r' else False


def get_only_r_oids(filepath):
    file = open(filepath)
    obj_list = json.load(file)
    file.close()

    oids = []
    tags = []
    for data in obj_list:
        if check_if_r(data['oid']):
            oids.append(data['oid'])
            tags.append(data['tags'])

    targets = [] # 1-artefact,  0-transient
    for tag_list in tags:
        if 'artefact' in tag_list:
            targets.append(1)
        else:
            targets.append(0)
    
    return oids, targets

oids, labels = get_only_r_oids('akb.ztf.snad.space.json')

In [ ]:
for oid, label in zip(oids[1644:], labels[1644:]):
    if str(oid)[0] == '1':
        field = str(oid)[:4]
    else:
        field = str(oid)[:3]

    file_types = zip(['feature', 'feature', 'oid'], ['dat', 'name', 'dat'])
    for file_type in file_types:
        url = f'https://sai.snad.space/tmp/features/{file_type[0]}_{field}.{file_type[1]}'
        with requests.get(url) as response:
            response.raise_for_status()
            open(f'akb_features/{file_type[0]}_{field}.{file_type[1]}', 'wb').write(response.content)
